In [ ]:
root_dir = '/home/nasha_voxelsafety_com/datasets/rite_hite/v1.0'

In [ ]:
import numpy as np
import torch

In [ ]:
torch.__version__

## Select subset of dataset

In [ ]:
import os
import numpy as np
source_dir = '/home/nasha_voxelsafety_com/datasets/dixieline/v1.0/'
dest_dir = '/home/nasha_voxelsafety_com/datasets/dixieline/v1.1/'
klasses = ['vest', 'no_vest']
for klass in klasses:
    vest_class = (os.listdir(f'{source_dir}{klass}/'))
    print(len(vest_class))
    dataset_size = len(vest_class)
    selected = np.random.choice(dataset_size, 3500)
    selected_filename = list(np.array(vest_class)[list(selected)])

    for f in selected_filename:
        if not os.path.exists(f"{dest_dir}{klass}/"):
            os.makedirs(f"{dest_dir}{klass}/")
        os.system(f"cp {source_dir}{klass}/{f} {dest_dir}{klass}/{f}")

In [ ]:
len(selected_filename)

## Split into Train and Test Folders

In [ ]:
root_dir = '/home/nasha_voxelsafety_com/datasets/dixieline/v1.2/'
new_dir = '/home/nasha_voxelsafety_com/datasets/dixieline/v1.3/'
site_name = 'dixieline'
# get the list 
filenames = []
for klass in os.listdir(root_dir):
    for f in os.listdir(f"{root_dir}{klass}"):
        filenames.append(("-").join(f.strip(f"{site_name}-").strip(".jpg").split("-")[:-2]))
    

In [ ]:
unique_incidents = np.unique(np.array(filenames))
from sklearn.model_selection import train_test_split
train_ids, test_ids = train_test_split(unique_incidents,test_size=0.2, random_state=42)
for klass in os.listdir(root_dir):
    for f in os.listdir(f"{root_dir}{klass}"):
        
        
        incident_id = ("-").join(f.strip(f"{site_name}-").strip(".jpg").split("-")[:-2])
        #train/test split
        if incident_id in train_ids:
            train_dir =  f"{new_dir}train/{klass}/"
            if not os.path.exists(train_dir):
                os.makedirs(train_dir)
            
            print(f"cp {root_dir}{klass}/{f} {train_dir}{f}") 
            os.system(f"cp {root_dir}{klass}/{f} {train_dir}{f}")
        elif incident_id in test_ids:
            test_dir = f"{new_dir}test/{klass}/"
            if not os.path.exists(test_dir):
                os.makedirs(test_dir)
            print(f"cp {root_dir}{klass}/{f} {test_dir}{f}")
            os.system(f"cp {root_dir}{klass}/{f} {test_dir}{f}")


## Split into Train and Test Pytorch Datasets (run once)

In [ ]:
from torchvision import datasets, transforms
dataset = datasets.ImageFolder(root_dir,transform=transforms.Compose([transforms.Resize(224),transforms.GaussianBlur(kernel_size=3)]))
indices = list(range(0,int(dataset.__len__()),1))
np.random.shuffle(indices)
split_1 = int(np.floor(0.6 * len(indices)))
split_2 = int(np.floor(0.4 * len(indices)))
print(split_1,split_1+split_2)
train_idx, test_idx, rand_idx = indices[:split_1], indices[split_1:split_1+split_2],indices[split_1+split_2:]
train_orig = torch.utils.data.Subset(dataset, train_idx)
test_orig = torch.utils.data.Subset(dataset, test_idx)
rand_orig = torch.utils.data.Subset(dataset, rand_idx)
print(len(indices))


In [ ]:
import os
train_names = np.array(dataset.samples)[train_idx,0]
test_names = np.array(dataset.samples)[test_idx,0]
for im in train_names:
    print(im)
    im_name = os.path.basename(im)
    destination = "/home/nasha_voxelsafety_com/datasets/rite_hite/v1.1/train"
    if not os.path.exists(destination):
        os.makedirs(destination+"vest")
        os.makedirs(destination+"no_vest")
        
    klass = "no_vest" if "no_vest" in im else "vest"
    os.system(f"cp {im} {destination}/{klass}/{im_name}")
for im in test_names:
    im_name = os.path.basename(im)
    destination = "/home/nasha_voxelsafety_com/datasets/rite_hite/v1.1/test"
    if not os.path.exists(destination):
        os.makedirs(destination+"vest")
        os.makedirs(destination+"no_vest")
    klass = "no_vest" if "no_vest" in im else "vest"
    os.system(f"cp {im} {destination}/{klass}/{im_name}")

In [ ]:
import os
with open('/home/nasha_voxelsafety_com/datasets/images.txt') as f:
    train_names = f.readlines()
source = "/home/nasha_voxelsafety_com/datasets/vestdataset/vest"
destination = "/home/nasha_voxelsafety_com/datasets/uscold/v1.0/train"
for im in train_names:
    if not os.path.exists(destination+"vest"):
        os.makedirs(destination+"vest")
    print(f"cp {source}/{im} {destination}/{im}")
    os.system(f"cp {source}/{im.strip()} {destination}/vest/{im}")


### Training on train directory

In [ ]:
root_dir_0 = '/home/nasha_voxelsafety_com/datasets/vest-dataset-clean'
root_dir_1 = f"{new_dir}train"

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import ViTFeatureExtractor
import os
from os import walk
import numpy as np
class MapDataset(Dataset):

    def __init__(self, dataset, map_fn):
        self.dataset = dataset
        self.map = map_fn
        
    def __getitem__(self, index):
        if self.map:
            x = self.map(self.dataset[index][0])
        else:     
            x = self.dataset[index][0]  # image
        y = self.dataset[index][1]   # label      
        return x, y

    def __len__(self):
        return len(self.dataset)

In [ ]:
from torchvision import datasets, transforms
import torch
def create_datasets(root_dir):
    dataset = datasets.ImageFolder(root_dir)
    class_names = dataset.classes
    idx2class = {v: k for k, v in dataset.class_to_idx.items()}
    class_names = dataset.classes
    num_train = len(dataset)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(0.2 * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]
    train_orig = torch.utils.data.Subset(dataset, train_idx)
    val_orig = torch.utils.data.Subset(dataset, valid_idx)
    feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k', return_tesnor = "pt")
    train = MapDataset(train_orig, feature_extractor)
    val =  MapDataset(val_orig, feature_extractor)
    return dataset, train, val

In [ ]:
from collections import Counter
def get_class_count(dataset, train, val):
    print(dataset.class_to_idx)
    idx2class = {v: k for k, v in dataset.class_to_idx.items()}
    class_count =  dict(Counter(dataset.targets))
    val_count = dict(Counter(list(np.array(dataset.targets)[val.dataset.indices])))
    train_count = dict(Counter(list(np.array(dataset.targets)[train.dataset.indices])))
    return class_count, val_count, train_count

In [ ]:
def get_train_weights(dataset,train,train_count):
    target_list_train = list(np.array(dataset.targets)[train.dataset.indices])
    class_count_train = [i for _,i in sorted(train_count.items())]
    class_weights_train = 1./torch.tensor(class_count_train, dtype=torch.float) 
    class_weights_train_all = class_weights_train[target_list_train]
    return class_weights_train_all

In [ ]:
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import RandomSampler

def get_samplers(class_weights_train_all,num_samples_train, num_samples_val):
    weighted_sampler_train = WeightedRandomSampler(
        weights=class_weights_train_all,
        num_samples=num_samples_train,
        replacement=True
    )
    random_sampler_val = RandomSampler(data_source=val,replacement=True,num_samples=num_samples_val)
    return weighted_sampler_train, random_sampler_val

In [ ]:
def get_dataloaders(train,val,batch_size,weighted_sampler_train,random_sampler_val):
    dataloaders = {
        'train': DataLoader(train, batch_size=batch_size, num_workers=8, sampler=weighted_sampler_train),
        'val': DataLoader(val, batch_size=batch_size, num_workers=8,sampler=random_sampler_val),
    }
    return dataloaders

In [ ]:
dataloaders = {}
# site_name = 'laredo-2'
num_samples_train=4441 
num_samples_val=1110
# for i, root_dir in enumerate([root_dir_1,root_dir_0]):
for i, root_dir in enumerate([root_dir_1]):
    dataset, train, val = create_datasets(root_dir)


    class_count, val_count, train_count = get_class_count(dataset,train,val)
    print(class_count, val_count, train_count, train.__len__(), val.__len__())

    class_weights_train_all = get_train_weights(dataset,train, train_count)

    weighted_sampler_train, random_sampler_val = get_samplers(class_weights_train_all,num_samples_train=4441, num_samples_val=1110)
    b = (class_weights_train_all) < max((class_weights_train_all))
    print(len(b.nonzero()))

    batch_size =32
    source = site_name if i==0 else "laredo"
    dataloaders[source] = get_dataloaders(train,val,batch_size,weighted_sampler_train,random_sampler_val)

In [ ]:
import matplotlib.pyplot as plt
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
class_names = ["no_vest", "vest"]
import torchvision
inputs, classes = next(iter(dataloaders[site_name]['train']))
inputs = inputs['pixel_values'][0]
print(classes.shape)
out = torchvision.utils.make_grid(inputs[0:16])
imshow(out, title=[class_names[x] for x in classes[0:16]])

In [ ]:
model_name_or_path = 'google/vit-base-patch16-224-in21k'

In [ ]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(dataset.classes),
    id2label={str(i): c for i, c in enumerate(dataset.classes)},
    label2id={c: str(i) for i, c in enumerate(dataset.classes)}
)

In [ ]:
from torch import nn
import wandb
from torch.optim import lr_scheduler
import time
import copy
from datetime import datetime


def train_model(model, config, dataloaders, dataset_sizes, optimizer):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = nn.CrossEntropyLoss() 
    model_path = '/home/nasha_voxelsafety_com/voxel/experimental/nasha/ppe/models/'
    
    wandb.init(project =  "vit_classification", entity = "voxel-wandb", config = config, tags = [config['tags']])
    config = wandb.config
    scheduler = lr_scheduler.StepLR(optimizer, step_size=config['step'], gamma=0.1)
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())   
    best_f1 = 0
    model.to(device)
    today_date = datetime.today().strftime('%Y-%m-%d')
    for epoch in range(config.num_epochs):
        print('Epoch {}/{}'.format(epoch, config.num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            running_tp = 0
            running_fp = 0
            running_fn = 0
            #running_tn = 0

            # Iterate over data.
            
#             for (input_0,labels_0), (input_1,labels_1) in zip(dataloaders["laredo"][phase],dataloaders[site_name][phase]):
#                 inputs = torch.cat((input_0['pixel_values'][0],input_1['pixel_values'][0]), 0)
#                 labels = torch.cat((labels_0,labels_1))
            for inputs, labels in dataloaders[site_name][phase]:
                inputs = inputs['pixel_values'][0]
                inputs = inputs.to(device)
                labels = labels.to(device).view(-1)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs = outputs.logits
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_tp += torch.sum(np.logical_and(preds.cpu(),labels.data.cpu())).cuda()
                running_fp += torch.sum(np.greater(preds.cpu(),labels.data.cpu())).cuda()
                running_fn += torch.sum(np.less(preds.cpu(),labels.data.cpu())).cuda()                

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase] 

            epoch_precision = running_tp/(running_tp+running_fp)
            epoch_recall = running_tp/(running_tp+running_fn)
            epoch_f1 = 2*(epoch_precision*epoch_recall)/(epoch_precision+epoch_recall)
            if phase == 'val':
                print("val loss", epoch_loss)
                print("val_f1", epoch_f1)
                wandb.log({"val loss":epoch_loss})
                wandb.log({"val_f1":epoch_f1})
            if phase == 'train':
                print("train loss", epoch_loss)
                wandb.log({"train loss":epoch_loss})

            print('{} Loss: {:.4f} Acc: {:.4f} Precision: {:.4f} Recall: {:.4f} F1: {:.4f}'.format(
                phase, epoch_loss, epoch_acc,epoch_precision,epoch_recall, epoch_f1))

            # deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_f1 = epoch_f1
                best_model_wts = copy.deepcopy(model.state_dict())
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_f1))


    
    # load best model weights
    model.load_state_dict(best_model_wts)

    #save this model

    torch.save(model.state_dict(), f'{model_path}/voxel_safetyvest_{config.tags}_{today_date}.pth')
    wandb.join()
    return model



In [ ]:
import transformers
transformers.__version__

In [ ]:
config = {
        'num_epochs': 10,
        'learning_rate': 1e-4,
        'step':10,
        'tags': f"vit_{site_name}"
}

optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

train_model(model, config, dataloaders, {"train":num_samples_train ,"val":num_samples_val}, optimizer)


## Testing

In [ ]:
model_name = '/home/nasha_voxelsafety_com/voxel/experimental/nasha/ppe/models/voxel_safetyvest_vit_dixieline_2022-09-16.pth'
model.load_state_dict(torch.load(model_name))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# Copyright 2020-2021 Voxel Labs, Inc.
# All rights reserved.
#
# This document may not be reproduced, republished, distributed, transmitted,
# displayed, broadcast or otherwise exploited in any manner without the express
# prior written permission of Voxel Labs, Inc. The receipt or possession of this
# document does not convey any rights to reproduce, disclose, or distribute its
# contents, or to manufacture, use, or sell anything that it may describe, in
# whole or in part.
#
import os

import numpy as np
import torch
import wandb
from sklearn.metrics import precision_recall_fscore_support
from torchvision import datasets, transforms
from PIL import Image

testdir = f"{new_dir}test"
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k', return_tesnor = "pt")


class TestClassifier:
    TEST_TABLE_NAME = "test_results_images"
    RESULT_TABLE_NAME = "test_results_metrics"
    PROJECT_NAME = "ppe_classification_sv_debug"

    def __init__(
        self,
        labels=["no_vest", "vest"],
        model_path="/home/reza_voxelsafety_com/voxel/experimental/reza/ppe_classification/models/voxel_safety_vest_rite_classifier_attention_resnet_focal_loss_gamma1.5_WS_2022-04-14.pth",
        model_name="vit_vest_classifier",
        test_dir="/home/reza_voxelsafety_com/experiments/ppe_classifaction/data/scenario_test",
        tags = ""
    ):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
        self.MODEL_NAME = model_name
        self.TEST_DIR = test_dir
        self.model = model
        self.class_labels = labels
        self.results = None
        self.test_dataloaders = None
        wandb.init(
            project=self.PROJECT_NAME,
            job_type="inference",
            entity="voxel-wandb",
            tags = tags,
        )

    def _input_to_image(self, inp):
        inp = inp.numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        inp = std * inp + mean
        inp = np.clip(inp, 0, 1)
        return inp

    def _calculate_metrics(self, pred, target, threshold=0.5):
        pred_flat = np.argmax(pred, 1)

        return {
            "metrics": precision_recall_fscore_support(
                y_true=target, y_pred=pred_flat
            ),
        }

    def load_dataset(self):

        test_datasets = {
            x: datasets.ImageFolder(
                os.path.join(self.TEST_DIR)
            )
            for x in ["test"]
        }
        self.names = test_datasets["test"].samples
        test_datasets['test'] =  MapDataset(test_datasets['test'], feature_extractor)
        self.test_dataloaders = {
            x: torch.utils.data.DataLoader(
                test_datasets[x], batch_size=1, num_workers=8, shuffle=False
            )
            for x in ["test"]
        }

    def visualize_predictions(self):
        columns = ["names", "Images", "Prediction", "Ground Truth"]
        for klass in self.class_labels:
            columns.append("score_" + klass)
        test_dt = wandb.Table(columns=columns)
        model_result = []
        targets = []
        #i, (images, labels) in enumerate(test_loader, 0):
        for i, (inputs, labels) in enumerate(self.test_dataloaders["test"], 0):

            name = self.names[i][0]
            image = Image.open(name)
            w, h = image.size
            if w * h < 2500:
                print("small image")
                continue
            name_img = name.split('/')[-1]
            inputs = inputs['pixel_values'][0]
            inputs = inputs.to(self.device)
            labels = labels.to(self.device)
            pred = self.model(inputs)
            pred = pred.logits
            model_result.extend(pred.cpu().detach().numpy())
            targets.extend(labels.cpu().numpy())

#             pred = pred.data.cpu().numpy()
            guess = self.class_labels[torch.argmax(pred)]

            row = [
                name_img,
                wandb.Image(self._input_to_image(inputs[0].cpu())),
                guess,
                self.class_labels[labels],
            ]

            for c_i in pred[0].tolist():
                row.append(np.round(c_i, 4))
            test_dt.add_data(*row)
        self.results = self._calculate_metrics(
            np.array(model_result), np.array(targets)
        )
        wandb.log({self.TEST_TABLE_NAME: test_dt})

    def log_metrics(self):
        metrics_table = wandb.Table(
            columns=["Class", "Precision", "Recall", "F1", "Support"]
        )

        for i in range(len(self.class_labels)):
            metrics_table.add_data(
                self.class_labels[i],
                self.results["metrics"][0][i],
                self.results["metrics"][1][i],
                self.results["metrics"][2][i],
                self.results["metrics"][3][i],
            )

        wandb.log({self.RESULT_TABLE_NAME: metrics_table})
        wandb.join()



test_classifier = TestClassifier(test_dir = testdir, model_path=model_name, tags = ["model:dixieline","data:dixieline_test"])
    # load the dataset
test_classifier.load_dataset()
    # run model on dataset and upload visuals to wandb
test_classifier.visualize_predictions()
    # log the overall metrics of the classifier
test_classifier.log_metrics()

In [ ]:
testdir = "/home/nasha_voxelsafety_com/datasets/clean_test/uscold_vest"
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k', return_tesnor = "pt")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from PIL import Image
TP_n = 0
FN_n = 0
TP = 0
FN = 0
for klass in os.listdir(testdir):
    klasspath = os.path.join(testdir,klass)
    filenames = [os.path.join(klasspath, file) for file in os.listdir(klasspath) if not file.startswith('.')]

    for filename in filenames:
        image = Image.open(filename)
        temp = feature_extractor(image, 'pt')
        image_tensor = torch.tensor(temp['pixel_values'][0])
        image_tensor = image_tensor.unsqueeze(0).to(device)
        outputs = model(pixel_values=image_tensor)
        temp = torch.argmax(outputs.logits)
        if klass == "vest":
            if temp == 1:
                TP = TP + 1
            elif temp==0:
                FN = FN + 1
        elif klass == "no_vest":
            if temp == 0:
                TP_n = TP_n + 1
            elif temp == 1:
                FN_n = FN_n + 1
            

In [ ]:
import pandas as pd
prec_vest = 100 * (TP/(TP + FN_n ))
recall_vest = 100 * (TP/(TP + FN ))
prec_novest = 100 * (TP_n/(TP_n + FN ))
recal_novest = 100 * (TP_n/(TP_n + FN_n ))
f1_vest = 2 * prec_vest * recall_vest / (recall_vest + prec_vest)
f1_novest = 2 * prec_novest * recal_novest / (recal_novest + prec_novest)
# initialize list of lists
data = [['vest', prec_vest, recall_vest, f1_vest], ['no_vest', prec_novest, recal_novest, f1_novest]]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['class', 'prec', 'recall', 'f1'])

df

In [ ]:
from transformers import (
    ViTConfig,
    ViTFeatureExtractor,
    ViTForImageClassification,
)
device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )

configuration = ViTConfig()
configuration.num_labels = 2
model = ViTForImageClassification(
    configuration
)
model.load_state_dict(
    torch.load('/home/reza_voxelsafety_com/voxel/experimental/reza/ppe_classification/models/voxel_safetyvest_vit_2022-04-18.pth')
)
model = model.to(
    device
)

In [ ]:
wandb.init(
            project="vest_scenario",
            job_type="inference",
            entity="voxel-wandb",
        )

TEST_TABLE_NAME = "test_results_images"
columns = ["names", "Images", "Prediction"]
labels=["no_vest", "vest"]
for klass in labels:
    columns.append("score_" + klass)
test_dt = wandb.Table(columns=columns)
dir = '/home/reza_voxelsafety_com/voxel/experimental/reza/data'
from PIL import Image
names = [os.path.join(dir, file) for file in os.listdir(dir) if not file.startswith('.')]
for k in range(len(names)):
    image = Image.open(names[k])
    temp = feature_extractor(image, 'pt')
    image_tensor = torch.tensor(temp['pixel_values'][0])
    image_tensor = image_tensor.unsqueeze(0).to(device)
    outputs = model(pixel_values=image_tensor)
    temp = torch.argmax(outputs.logits)
    guess = labels[temp.item()]
    row = [names[k],
            wandb.Image(image),
            guess,]
    for c_i in outputs.logits.data.cpu().numpy().tolist():
        for c in c_i:
            row.append(np.round(c, 4))
    test_dt.add_data(*row)
wandb.log({TEST_TABLE_NAME: test_dt})